In [1]:
# =================================================================
# File: 4.rag_with_pinecone.ipynb
# Purpose: 로컬 폴더의 모든 PDF를 읽어 Pinecone에 저장하고 테스트
# =================================================================

import os
import time
import glob
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

c:\Users\lw105\OneDrive\바탕 화면\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. 환경변수 로드
load_dotenv()

# 키 확인
if not os.getenv("OPENAI_API_KEY") or not os.getenv("PINECONE_API_KEY"):
    print("🚨 .env 파일을 확인해주세요! 키가 없습니다.")
else:
    print("✅ 환경 변수 로드 완료")

# 2. 설정
INDEX_NAME = "rm-project-index"  # Pinecone에서 만든 인덱스 이름과 같아야 함

✅ 환경 변수 로드 완료


In [ ]:
# 3. 현재 폴더의 모든 PDF 찾기
pdf_files = glob.glob("*.pdf")
print(f"📂 발견된 PDF 파일: {pdf_files}")

if not pdf_files:
    print("🚨 폴더에 PDF 파일이 없습니다. 파일을 넣어주세요.")
else:
    # 4. 문서 로드 및 전처리
    documents = []

    for file_path in pdf_files:
        try:
            loader = PyPDFLoader(file_path)
            docs = loader.load()

            # 메타데이터에 파일명(출처) 추가 -> 답변할 때 "어느 문서에서 봤다"고 말하기 위함
            for doc in docs:
                doc.metadata["source"] = file_path

            documents.extend(docs)
            print(f"   ✅ 로드 성공: {file_path}")
        except Exception as e:
            print(f"   ❌ 로드 실패: {file_path} - {e}")

    if documents:
        # 청킹 (법률 문서는 호흡이 길어서 1000자 단위 추천)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200
        )
        texts = text_splitter.split_documents(documents)
        print(f"📊 총 {len(texts)}개의 지식 청크 생성됨")

        # 5. Pinecone에 업로드
        embeddings = OpenAIEmbeddings()

        print(f"🚀 Pinecone Index '{INDEX_NAME}'에 저장 중...")
        docsearch = PineconeVectorStore.from_documents(
            texts, embeddings, index_name=INDEX_NAME
        )
        print("✅ 저장 완료! 잠시 대기(3초)...")
        time.sleep(3)

        # 6. RAG 테스트 (RM 페르소나)
        template = """
        당신은 '불공정 약관 심사 전문 AI 변호사 RM'입니다.
        데이터베이스에는 [법적 기준]과 [기업 약관]이 섞여 있습니다.

        [임무]
        반드시 **[법적 기준 문서]**의 내용을 근거로 **[기업 약관]**이 공정한지 판결하세요.
        
        [답변 형식]
        - 결론: (공정/불공정)
        - 근거: 법령 내용 인용
        - 출처: **[출처: 파일명]** 표기 필수

        Context: {context}
        Question: {question}
        Answer:
        """
        PROMPT = PromptTemplate(
            template=template, input_variables=["context", "question"]
        )

        llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

        qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=docsearch.as_retriever(search_kwargs={"k": 5}),
            chain_type_kwargs={"prompt": PROMPT},
        )

        # 질문 테스트
        query = "넷플릭스 약관이랑 소비자분쟁해결기준 비교했을 때 환불 규정 문제 있어?"
        print(f"\n🔍 Test Question: {query}")
        print("-" * 50)
        print(qa.invoke({"query": query})["result"])

📂 발견된 PDF 파일: ['google개인정보처리방침.pdf', 'google서비스약관.pdf', '개인정보 처리방침 작성지침(2025.4.).pdf', '네이버.pdf', '넷플릭스.pdf', '소비자분쟁해결기준 개정안(2025).pdf', '약관의 규제에 관한 법률(법률)(제20239호)(20240807).pdf']
   ✅ 로드 성공: google개인정보처리방침.pdf
   ✅ 로드 성공: google서비스약관.pdf
   ✅ 로드 성공: 개인정보 처리방침 작성지침(2025.4.).pdf
   ✅ 로드 성공: 네이버.pdf
   ✅ 로드 성공: 넷플릭스.pdf
   ✅ 로드 성공: 소비자분쟁해결기준 개정안(2025).pdf
   ✅ 로드 성공: 약관의 규제에 관한 법률(법률)(제20239호)(20240807).pdf
📊 총 536개의 지식 청크 생성됨
